In [1]:
### Import libraries ----
import pandas as pd
import numpy as np
import sklearn.linear_model as linear_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

In [2]:
### Generate data ----
#!Python 1_Get_UCDP.py

In [3]:
### Load data ----
df_ucdp = pd.read_csv('ucdp_month.csv')
df_ucdp = df_ucdp.set_index("date")
df_ucdp.head(5)

,Afghanistan,Albania,Algeria,Angola,Argentina,Armenia,Australia,Austria,Azerbaijan,Bahrain,...,Uganda,Ukraine,United Arab Emirates,United Kingdom,United States of America,Uzbekistan,Venezuela,Yemen (North Yemen),Zambia,Zimbabwe (Rhodesia)
date,,,,,,,,,,,,,,,,,,,,,
1989-01,1298.788462,0,0.0,300.273810,0,0.0,0,0,0.0,0,...,0.000000,0.0,0,3.0,0,0.0,0.000000,0.0,0.0,0.0
1989-02,198.715385,0,0.0,427.809524,0,0.0,0,0,0.0,0,...,0.000000,0.0,0,5.0,0,0.0,21.897727,0.0,0.0,0.0
1989-03,2211.050962,0,0.0,415.261905,0,0.0,0,0,0.0,0,...,0.000000,0.0,0,5.0,0,0.0,22.505682,0.0,0.0,10.5
1989-04,510.730769,0,0.0,177.857143,0,0.0,0,0,0.0,0,...,97.333333,0.0,0,0.0,0,0.0,0.000000,0.0,0.0,0.0
1989-05,821.833544,0,0.0,278.592437,0,0.0,0,0,0.0,0,...,593.126667,0.0,0,1.0,0,0.0,0.000000,0.0,4.0,0.0


In [4]:
### Prepare data -----
number_s=11          # 10 months sequences

# Normalization
scaler = MinMaxScaler(feature_range=(0,1))
df = pd.DataFrame(scaler.fit_transform(df_ucdp))

In [5]:
# Creation of the sequences
ts_seq=[]
for col in range(len(df.columns)):
    for i in range(number_s-1,len(df)):
        ts_seq.append(df.iloc[i-number_s+1:i+1,col])

In [6]:
# Creation of trainset, validation and testet
ts_seq=np.array(ts_seq)
ts_seq_l= ts_seq.reshape(len(df.columns),len(df.index)-number_s+1,number_s)

ts_seq_learn=ts_seq_l[:,:int(0.6*len(df)),:]
ts_seq_learn=ts_seq_learn.reshape(ts_seq_learn.shape[0]*ts_seq_learn.shape[1],number_s)

ts_seq_val=ts_seq_l[:,int(0.6*len(df)):int(0.8*len(df)),:]
ts_seq_val=ts_seq_val.reshape(ts_seq_val.shape[0]*ts_seq_val.shape[1],number_s)

ts_seq_test=ts_seq_l[:,int(0.8*len(df)):,:]
ts_seq_test=ts_seq_test.reshape(ts_seq_test.shape[0]*ts_seq_test.shape[1],number_s)

In [7]:
# Creation of input/output for each set
train_x = ts_seq_learn[:,:-1]
train_y = ts_seq_learn[:,-1]
val_x = ts_seq_val[:,:-1]
val_y = ts_seq_val[:,-1]
test_x = ts_seq_test[:,:-1]
test_y = ts_seq_test[:,-1]

In [23]:
### Length of partitions ----
print(len(train_x))
print(len(val_x))
print(len(test_x))

28560
9480
8400


In [8]:
# Hyperparameter tuning -----

tuning = pd.DataFrame(columns=["alpha", "mse", "wmse"])

for a in [0.1, 2, 3, 5, 10, 50, 200, 1000, 2000]:
    model = linear_model.Lasso(max_iter=2000,alpha=1/(2*a)).fit(train_x, train_y)
    pred = model.predict(val_x)
    mse = mean_squared_error(val_y, pred)
    wmse =  mean_squared_error(val_y,pred,sample_weight=val_y+1)
    
    # Save results
    para = pd.DataFrame({"alpha": a,
                         "mse": mse, 
                         "wmse": wmse},
                         index=[a])
    tuning = tuning.append(para)
    tuning.reset_index(drop=True, inplace=True)

tuning.to_latex(f'tuning_lasso.tex', index=False)
tuning

,alpha,mse,wmse
0,0.1,0.008102,0.012932
1,2.0,0.008102,0.012932
2,3.0,0.008102,0.012932
3,5.0,0.008102,0.012932
4,10.0,0.008102,0.012932
5,50.0,0.008102,0.012932
6,200.0,0.005524,0.008928
7,1000.0,0.003727,0.006017
8,2000.0,0.003635,0.005842


In [9]:
### Final model -----
model = linear_model.Lasso(max_iter=2000,alpha=1/(2*1000)).fit(train_x, train_y)
pred = model.predict(test_x)
mse = mean_squared_error(test_y,pred)
print(mse)
weighted_mse =  mean_squared_error(test_y,pred,sample_weight=test_y+1)
print(weighted_mse)

0.0048207315080313895
0.007719226664671329


In [10]:
### Convert back to original dataframe ----
df_lasso = pred.reshape((len(df.iloc[0,:]),int(len(ts_seq_test[:,:-1])/len(df.iloc[0,:]))))
df_lasso = df_lasso.T
df_lasso = pd.DataFrame(df_lasso)
df_lasso = pd.DataFrame(scaler.inverse_transform(df_lasso))
df_lasso.columns = df_ucdp.columns
df_lasso.index=df_ucdp.index[-70:]
df_lasso.to_csv('preds_lasso.csv', index=False)
df_lasso.head(10)

,Afghanistan,Albania,Algeria,Angola,Argentina,Armenia,Australia,Austria,Azerbaijan,Bahrain,...,Uganda,Ukraine,United Arab Emirates,United Kingdom,United States of America,Uzbekistan,Venezuela,Yemen (North Yemen),Zambia,Zimbabwe (Rhodesia)
date,,,,,,,,,,,,,,,,,,,,,
2016-04,1415.731326,0.042177,16.946770,34.434691,0.604536,0.873303,0.014059,0.028118,42.591886,0.056236,...,8.403000,41.548661,0.007029,0.365533,21.570696,2.434545,4.084575,455.512543,0.14059,1.124718
2016-05,1602.394077,0.042177,16.990898,34.481477,0.604536,0.914229,0.014059,0.028118,88.996952,0.056236,...,8.662566,41.573781,0.007029,0.365533,21.517788,2.434545,4.358390,378.303374,0.14059,1.124718
2016-06,1799.340304,0.042177,19.892778,34.543286,0.604536,0.894975,0.014059,0.028118,60.485396,0.056236,...,6.535539,43.862054,0.007029,0.365533,21.421828,2.434545,3.577839,360.429328,0.14059,1.124718
2016-07,1667.574099,0.042177,18.232363,35.732147,0.604536,0.879593,0.014059,0.028118,53.527588,0.056236,...,5.973271,34.455733,0.007029,0.365533,21.414064,2.434545,3.783547,381.010836,0.14059,1.124718
2016-08,1625.941271,0.042177,14.063918,36.207089,0.604536,0.752155,0.014059,0.028118,48.763030,0.056236,...,5.618882,40.963555,0.007029,0.365533,21.557304,2.434545,4.117825,339.905005,0.14059,1.124718
2016-09,1726.595840,0.042177,12.482952,35.677474,0.604536,0.752155,0.014059,0.028118,47.725879,0.056236,...,5.526633,34.949455,0.007029,0.365533,21.489914,2.434545,3.765463,346.092969,0.14059,1.124718
2016-10,1680.216511,0.042177,13.142932,36.575008,0.604536,0.752155,0.014059,0.028118,46.408433,0.056236,...,5.617349,29.968899,0.007029,0.365533,21.436078,2.434545,4.779100,289.980065,0.14059,1.124718
2016-11,2168.241883,0.042177,13.680645,35.634760,0.604536,0.752155,0.014059,0.028118,45.798036,0.056236,...,5.625247,39.439812,0.007029,0.365533,20.990045,2.434545,3.680470,336.323908,0.14059,1.124718
2016-12,1620.193767,0.042177,12.172563,35.971412,0.604536,0.752155,0.014059,0.028118,47.726236,0.056236,...,5.610270,30.954938,0.007029,0.365533,20.990045,2.434545,6.140706,409.891618,0.14059,1.124718


In [14]:
### Convert test df to original dataframe format and save ----
df_test = test_y.reshape((len(df.iloc[0,:]),int(len(ts_seq_test[:,:-1])/len(df.iloc[0,:]))))
df_test = df_test.T
df_test = pd.DataFrame(df_test)
df_test = pd.DataFrame(scaler.inverse_transform(df_test))
df_test.columns = df_ucdp.columns
df_test.index=df_ucdp.index[-70:]
df_test.to_csv('df_test.csv', index=True)
df_test.head(10)

,Afghanistan,Albania,Algeria,Angola,Argentina,Armenia,Australia,Austria,Azerbaijan,Bahrain,...,Uganda,Ukraine,United Arab Emirates,United Kingdom,United States of America,Uzbekistan,Venezuela,Yemen (North Yemen),Zambia,Zimbabwe (Rhodesia)
date,,,,,,,,,,,,,,,,,,,,,
2016-04,2389.000000,0.0,15.0,1.315068,0.0,0.0,0.0,0.0,161.750000,0.0,...,7.752809,47.666667,0.0,0.0,0.0,0.0,5.0,296.476923,0.0,0.0
2016-05,2801.833333,0.0,24.0,1.358904,0.0,0.0,0.0,0.0,5.000000,0.0,...,0.000000,62.333333,0.0,0.0,0.0,0.0,2.0,359.974359,0.0,0.0
2016-06,2201.900000,0.0,15.0,5.315068,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,30.166667,0.0,0.0,0.0,0.0,4.0,500.166667,0.0,0.0
2016-07,2206.773810,0.0,0.0,5.358904,0.0,0.0,0.0,0.0,1.000000,0.0,...,0.000000,58.342857,0.0,0.0,0.0,0.0,5.0,367.500000,0.0,0.0
2016-08,2728.502564,0.0,0.0,2.358904,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,30.000000,0.0,0.0,0.0,0.0,3.0,451.000000,0.0,0.0
2016-09,2271.791667,0.0,5.0,5.815068,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,16.000000,0.0,0.0,0.0,0.0,6.5,258.000000,0.0,0.0
2016-10,3964.097009,0.0,6.0,1.358904,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,54.500000,0.0,0.0,0.0,0.0,2.0,486.000000,0.0,0.0
2016-11,1382.900000,0.0,0.0,3.315068,0.0,0.0,0.0,0.0,1.000000,0.0,...,0.000000,12.428571,0.0,0.0,0.0,0.0,12.0,690.534483,0.0,0.0
2016-12,1273.900000,0.0,0.0,1.358904,0.0,4.0,0.0,0.0,0.000000,0.0,...,0.000000,6.000000,0.0,0.0,0.0,0.0,0.0,272.000000,0.0,0.0
